<a href="https://colab.research.google.com/github/lleiva25/Home_Sales/blob/main/Home_Sales_starter_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install findspark
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=8c53d462d6c908d8526d37a3284eba72aeff7618cc1ae3837f333b95b6dcd82f
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [4]:
# Import findspark and initialize.
import findspark
findspark.init()

In [5]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [6]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [9]:
# 2. Create a temporary view of the DataFrame.
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get('home_sales_revised.csv'), sep=",", header=True)
df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [55]:
from typing import ValuesView
from pyspark.sql.functions import mean as _mean
from pyspark.sql.functions import split

# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?

#split date column to have year, month and day in separate columns
df = df.withColumn('year', split(df['date'], '-').getItem(0))
df = df.withColumn('month', split(df['date'], '-').getItem(1))
df = df.withColumn('day', split(df['date'], '-').getItem(2))

#Filter dataframe by 4 bedrooms
filter_4bed_df = df.filter(df.bedrooms == 4)
print('='*100)
print('-'*20+'Show Filtered Data Frame'+'-'*20)
filter_4bed_df.show()
print('='*100)

#Create a view based on filtered database
filter_4bed_df.createOrReplaceTempView('4bedrooms')

#Initiate start time of execution
start_time = time.time()

#The view with out dataframe and run SQL that will average the price of 4 bedrooms by year
print('-'*13+'Show SQL View for AVG Price for 4 Bedrooms in Each Year'+'-'*13)
bedrooms4_avg_yr = spark.sql("""select year, ROUND(avg(price),2) AS average_price from 4bedrooms GROUP BY year ORDER BY year""").show(truncate=False)

#Show timing of the execution
print("--- %s seconds ---" % (time.time() - start_time))

--------------------Show Filtered Data Frame--------------------
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+----+-----+---+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|year|month|day|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+----+-----+---+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|2022|   04| 08|
|f233cb41-6f33-4b0...|2021-07-18|      2016|437375|       4|        3|       1704|   11721|     2|         0|  34|2021|   07| 18|
|2dc662fe-57b8-44b...|2020-04-20|      2016|417215|       4|        2|       2104|    8227|     2|         0|  48|2020|   04| 20|
|d5f2062a-8950-417...|2019-09-02|      2016|305980|       4|        2|       1799|   13004|     1|         0|  26|2019|   09| 02|
|f5e01433-f7b0-44e...|202

In [73]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?

#Filter dataframe by 3 bedrooms
filter_3bed_3bath_df = df.filter(df.bedrooms == 3).filter(df.bathrooms ==3)
print('='*100)
print('-'*20+'Show Filtered Data Frame'+'-'*20)
filter_3bed_3bath_df.show()
print('='*100)

#Create a view based on filtered database
filter_3bed_3bath_df.createOrReplaceTempView('3beds3baths')

#Initiate start time of execution
start_time = time.time()

#The view with out dataframe and run SQL that will average the price of 3 bed 3 baths by year
print('-'*13+'Show SQL View for AVG Price for 3 Bed 3 Baths in Each Year'+'-'*13)
bed_3_bath_avg_yr = spark.sql("""select date_built, ROUND(avg(price),2) AS average_price from 3beds3baths GROUP BY date_built ORDER BY date_built""").show(truncate=False)

#Show timing of the execution
print("--- %s seconds ---" % (time.time() - start_time))

--------------------Show Filtered Data Frame--------------------
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+----+-----+---+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|year|month|day|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+----+-----+---+
|e81aacfe-17fe-46b...|2020-06-16|      2016|181925|       3|        3|       2137|   11709|     2|         0|  22|2020|   06| 16|
|2ed8d509-7372-46d...|2021-08-06|      2015|258710|       3|        3|       1918|    9666|     1|         0|  25|2021|   08| 06|
|f876d86f-3c9f-42b...|2019-02-27|      2011|167864|       3|        3|       2471|   13924|     2|         0|  15|2019|   02| 27|
|941bad30-eb49-4a7...|2020-05-09|      2015|229896|       3|        3|       2197|    8641|     1|         0|   3|2020|   05| 09|
|ea620c7b-c2f7-4c6...|202

In [72]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?

#Filter dataframe by 3 bedrooms
filter_3bed_3bath_bighome_df = df.filter(df.bedrooms == 3).filter(df.bathrooms ==3).filter(df.floors == 2).filter(df.sqft_lot >= 2000)
print('='*100)
print('-'*20+'Show Filtered Data Frame'+'-'*20)
filter_3bed_3bath_bighome_df.show()
print('='*100)

#Create a view based on filtered database
filter_3bed_3bath_bighome_df.createOrReplaceTempView('3beds3bathsbighouse')

#Initiate start time of execution
start_time = time.time()

#The view with out dataframe and run SQL that will average the price of 3 bed 3 baths 2 floors and 2000sqft by year
print('-'*13+'Show SQL View for AVG Price for 3 Bed, 3 Baths, 2 Floors & 2,000 square feet in Each Year'+'-'*13)
bed_3_bath_big_house_avg_yr = spark.sql("""select date_built, ROUND(avg(price),2) AS average_price from 3beds3bathsbighouse GROUP BY date_built ORDER BY date_built""").show(truncate=False)

#Show timing of the execution
print("--- %s seconds ---" % (time.time() - start_time))

--------------------Show Filtered Data Frame--------------------
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+----+-----+---+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|year|month|day|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+----+-----+---+
|e81aacfe-17fe-46b...|2020-06-16|      2016|181925|       3|        3|       2137|   11709|     2|         0|  22|2020|   06| 16|
|f876d86f-3c9f-42b...|2019-02-27|      2011|167864|       3|        3|       2471|   13924|     2|         0|  15|2019|   02| 27|
|0cfe57f3-28c2-472...|2019-10-04|      2015|308313|       3|        3|       1960|    9453|     2|         0|   2|2019|   10| 04|
|4566cd2a-ac6e-435...|2019-07-15|      2016|177541|       3|        3|       2130|   10517|     2|         0|  25|2019|   07| 15|
|50873b8b-92b8-44c...|202

In [109]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating.
# Although this is a small dataset, determine the run time for this query.
view_df = df.filter(df.price >= 350000).filter(df.view != 'NA')

#Create a view based on filtered database
view_df.createOrReplaceTempView('views')

#Initiate start time of execution
start_time = time.time()

print('-'*13+'Show SQL View for AVG Price based on View Ratings'+'-'*13)
avg_view = spark.sql("""select view, ROUND(avg(price),2) AS average_price from views GROUP BY view ORDER BY view""").show(truncate=False)

print("--- %s seconds ---" % (time.time() - start_time))

-------------Show SQL View for AVG Price based on View Ratings-------------
+----+-------------+
|view|average_price|
+----+-------------+
|0   |403848.51    |
|1   |401044.25    |
|10  |401868.43    |
|100 |1026669.5    |
|11  |399548.12    |
|12  |401501.32    |
|13  |398917.98    |
|14  |398570.03    |
|15  |404673.3     |
|16  |399586.53    |
|17  |398474.49    |
|18  |399332.91    |
|19  |398953.17    |
|2   |397389.25    |
|20  |399522.81    |
|21  |399758.88    |
|22  |402022.68    |
|23  |403411.92    |
|24  |400284.92    |
|25  |401298.69    |
+----+-------------+
only showing top 20 rows

--- 0.4588649272918701 seconds ---


In [103]:
# 7. Cache the the temporary table home_sales.
#drop table if it already exists
spark.sql("DROP TABLE IF EXISTS home_sales")

#Create view
df.createOrReplaceTempView('home_sales')

#Check if table exists
spark.catalog.listTables()

#Cache the temporpary table
spark.sql('cache table home_sales')

DataFrame[]

In [104]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [108]:
# 9. Using the cached data, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the uncached runtime.

#Initiate start time of execution
start_time = time.time()

#The view with out dataframe and run SQL that will average the price by view
print('-'*13+'Show SQL View for AVG Price based on View Ratings'+'-'*13)
avg_view_cache = spark.sql("""select view, ROUND(avg(price),2) AS average_price from home_sales GROUP BY view ORDER BY view""").show(truncate=False)


print("--- %s seconds ---" % (time.time() - start_time))


-------------Show SQL View for AVG Price based on View Ratings-------------
+----+-------------+
|view|average_price|
+----+-------------+
|0   |285069.21    |
|1   |288248.11    |
|10  |289693.49    |
|100 |1026669.5    |
|11  |280356.07    |
|12  |286575.22    |
|13  |285849.95    |
|14  |282519.32    |
|15  |284907.04    |
|16  |291990.83    |
|17  |286601.22    |
|18  |287532.36    |
|19  |280347.62    |
|2   |287710.06    |
|20  |284820.43    |
|21  |283451.36    |
|22  |284908.42    |
|23  |286977.37    |
|24  |281878.86    |
|25  |288583.21    |
+----+-------------+
only showing top 20 rows

--- 0.4946770668029785 seconds ---


In [110]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").mode("overwrite").parquet("delayed_partitioned")

In [111]:
# 11. Read the formatted parquet data.
p_df_p=spark.read.parquet('delayed_partitioned')

In [114]:
# 12. Create a temporary table for the parquet data.
#drop table if it already exists
spark.sql("DROP TABLE IF EXISTS parquet_data")

#Create view
p_df_p.createOrReplaceTempView('parquet_data')

#Check if table exists
spark.catalog.listTables()

[Table(name='3beds3baths', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='3beds3bathsbighouse', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='4bedrooms', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='bedrooms', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='home_sales', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='home_sales_revised', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='parquet_data', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='views', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]

In [115]:
# 13. Using the parquet DataFrame, run the last query above, that calculates
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000.
# Determine the runtime and compare it to the cached runtime.

#Initiate start time of execution
start_time = time.time()

#The view with out dataframe and run SQL that will average the price by view
print('-'*13+'Show SQL View for AVG Price based on View Ratings Partition'+'-'*13)
avg_view_parquet = spark.sql("""select view, ROUND(avg(price),2) AS average_price from parquet_data GROUP BY view ORDER BY view""").show(truncate=False)


print("--- %s seconds ---" % (time.time() - start_time))


-------------Show SQL View for AVG Price based on View Ratings Partition-------------
+----+-------------+
|view|average_price|
+----+-------------+
|0   |285069.21    |
|1   |288248.11    |
|10  |289693.49    |
|100 |1026669.5    |
|11  |280356.07    |
|12  |286575.22    |
|13  |285849.95    |
|14  |282519.32    |
|15  |284907.04    |
|16  |291990.83    |
|17  |286601.22    |
|18  |287532.36    |
|19  |280347.62    |
|2   |287710.06    |
|20  |284820.43    |
|21  |283451.36    |
|22  |284908.42    |
|23  |286977.37    |
|24  |281878.86    |
|25  |288583.21    |
+----+-------------+
only showing top 20 rows

--- 1.18998384475708 seconds ---


In [116]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [117]:
# 15. Check if the home_sales is no longer cached

spark.catalog.isCached("home_sales")

False